In [1]:
import deap
import pandas as pd
import numpy as np
import random

In [ ]:
class Genetic:
    """ Wrapper class for deap genetic algorithms
    """
    
    def _init_(self, beginning_networks, data):
    """ Initialize genetic class
    
        :param: self: current genetic class
        :param: beginning_networks: first neural networks to start evolving
        :param: data: array/list/data structure of useful replay data
    """
        #our parameters
        self.finalCount = 10
            # How many individuals do we want in the end?
        #weight domain (for mutations)
        self.__weightMAX = 1.0
        self.__weightMIN = 0.0
        
        #deap algorithm arguments
        self.networks = beginning_networks
        self.data = data
        self.toolbox = Toolbox()
        self.tournsize = 2
            # number of parents selected from each generation
        self.genSize = 10
        self.cxpb = 1.0
            # probability of mating two individuals
        self.mutpb_indiv = 1.0
            # probability of mutating an individual
            # 1.0 because we want to maybe mutate a weight,
                # not maybe mutate an entire individual.
            # 1.0 means that no individual will be exempt
                # from its weights being mutated.
        self.mutpb_weight = 0.2
            # probability of mutating a weight
        self.ngen = 10000
            # 10000 is probably far too low/high.
                # I pulled it out of a hat.
            # TODO see if there's a standard value or
                # if we just need to do trial & error
                # to balance time & performance.
        
    def begin(self):
    """ 
        Toolbox needs aliases: mate, mutate, select, evaluate
    """
        self.toolbox.register("mate", ,)
            #TODO write custom crossover function
        self.toolbox.register("mutate"
            , mutate #defined below
            , mutProb=self.mutpb_weight
            , weightMIN=self.__weightMIN
            , weightMAX=self.__weightMAX )
        self.toolbox.register("select"
            , deap.tools.selTournament
                #...(individuals, k, tournsize, fit_attr='fitness')
            , k = self.genSize
            , tournsize = self.tournsize
            , fit_attr = 'fitness')
        self.toolbox.register("evaluate", ,)
            #TODO write custom evaluate function
        
        population = self.networks
        cxpb = self.cxpb
            # probability of mating two individuals
        mutpb = self.mutpb_indiv
            # probability of mutating an individual
        ngen = self.ngen
        halloffame = deap.tools.HallOfFame( self.finalCount )
            # Object that keeps track of the x best individuals.
            # x = self.finalCount = number of individuals we want in the end
        
        deap.algorithms.eaSimple(population, self.toolbox, cxpb, mutpb, ngen, halloffame)
        
        return halloffame
        
def mutate( individual, mutProb, weightMIN, weightMAX ):
"""
    Mutates an individual, in-place.
    For each weight, maybe assigns new value between
         [ weightMIN, weightMAX ].
    :param individual: the neural network to be mutated
    :param mutProb: the probability of mutating a given weight
    :param weightMIN: minimum value of a weight
    :param weightMAX: maximum value of a weight
"""
    LAYERS = individual.num_layers
        # TODO I prefer a get_num_layers to accessing
            # this member variable
        # TODO I think I saw a bug in get_weights in network.py.
            # Shouldn't an error be raised if layer >= self.num_layers
            # not layer > self.num_layers?
            # Jose is new to Python, so he could be wrong.
            # If he's wrong, change LAYERS-1 to LAYERS.
    
    #for all layers
    for l in range( 0, LAYERS-1 ):
        
        weights = individual.get_weights( l )
            # numpy array copy of this layer's weights
        
        #for all weights in this layer
        for w in range( 0, len(weights) ):
            if random.uniform(0,1) <= mutProb:
                #randomize the weight
                newWeights[w] = random.uniform( weightMIN, weightMAX )
                    #TODO IMPORTANT
                        # Jose doesn't know if we want integers
                        # or floating points. Jose doesn't know
                        # what the weight domain is.
        
        individual.set_weights( l, weights )
            # set weights for one layer